In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
data = pd.read_csv('data/train.csv')
print(data.shape) #(45000, 19)

(45000, 19)


In [3]:
print(data.isnull().sum())

Flight_ID                 0
Airline                3573
Departure_City          340
Arrival_City            186
Distance                 91
Departure_Time            0
Arrival_Time              0
Duration                  0
Aircraft_Type            43
Number_of_Stops           0
Day_of_Week             225
Month_of_Travel         267
Holiday_Season         9020
Demand                  317
Weather_Conditions      302
Passenger_Count           0
Promotion_Type        15215
Fuel_Price               90
Flight_Price              0
dtype: int64


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Flight_ID           45000 non-null  object 
 1   Airline             41427 non-null  object 
 2   Departure_City      44660 non-null  object 
 3   Arrival_City        44814 non-null  object 
 4   Distance            44909 non-null  float64
 5   Departure_Time      45000 non-null  object 
 6   Arrival_Time        45000 non-null  object 
 7   Duration            45000 non-null  float64
 8   Aircraft_Type       44957 non-null  object 
 9   Number_of_Stops     45000 non-null  int64  
 10  Day_of_Week         44775 non-null  object 
 11  Month_of_Travel     44733 non-null  object 
 12  Holiday_Season      35980 non-null  object 
 13  Demand              44683 non-null  object 
 14  Weather_Conditions  44698 non-null  object 
 15  Passenger_Count     45000 non-null  int64  
 16  Prom

In [5]:
print(data.describe())

           Distance      Duration  Number_of_Stops  Passenger_Count  \
count  44909.000000  45000.000000     45000.000000     45000.000000   
mean    5482.238727      7.852555         0.665667       174.928822   
std     2594.847860      3.292880         0.750578        72.372980   
min     1000.000000      1.280000         0.000000        50.000000   
25%     3230.000000      5.040000         0.000000       112.000000   
50%     5476.000000      7.840000         1.000000       175.000000   
75%     7725.000000     10.650000         1.000000       238.000000   
max    10000.000000     14.450000         3.000000       300.000000   

         Fuel_Price  Flight_Price  
count  44910.000000  45000.000000  
mean       0.850192    574.606270  
std        0.201736    160.239344  
min        0.500000    173.350000  
25%        0.680000    455.297500  
50%        0.850000    565.750000  
75%        1.020000    679.692500  
max        1.200000   1261.640000  


In [6]:
# Handle missing values
categorical_columns = ['Airline', 'Departure_City', 'Arrival_City', 'Demand', 'Aircraft_Type', 'Day_of_Week', 'Month_of_Travel','Holiday_Season','Weather_Conditions']

for column in categorical_columns:
    data[column].fillna('Missing', inplace=True)

numerical_columns = ['Distance', 'Fuel_Price']

for column in numerical_columns:
    data[column].fillna(data[column].median(), inplace=True)

data['Promotion_Type_Missing'] = data['Promotion_Type'].isnull().astype(int)

data['Fuel_Price'].fillna(data['Fuel_Price'].median(), inplace=True)

data.isnull().sum()


Flight_ID                     0
Airline                       0
Departure_City                0
Arrival_City                  0
Distance                      0
Departure_Time                0
Arrival_Time                  0
Duration                      0
Aircraft_Type                 0
Number_of_Stops               0
Day_of_Week                   0
Month_of_Travel               0
Holiday_Season                0
Demand                        0
Weather_Conditions            0
Passenger_Count               0
Promotion_Type            15215
Fuel_Price                    0
Flight_Price                  0
Promotion_Type_Missing        0
dtype: int64

In [7]:
# Combine 'Departure_City' and 'Arrival_City' to create a set of unique cities
unique_cities = set(data['Departure_City'].unique()) | set(data['Arrival_City'].unique())

In [8]:
# Create a mapping dictionary where each unique city is assigned a unique integer label
city_mapping = {city: label for label, city in enumerate(unique_cities)}

In [9]:
# Apply the mapping to both 'Departure_City' and 'Arrival_City'
data['Departure_City'] = data['Departure_City'].map(city_mapping)
data['Arrival_City'] = data['Arrival_City'].map(city_mapping)
data['Flight_ID'] = data['Flight_ID'].str.extract('(\d+)').astype(int)

In [10]:
data.head()

,Flight_ID,Airline,Departure_City,Arrival_City,Distance,Departure_Time,Arrival_Time,Duration,Aircraft_Type,Number_of_Stops,Day_of_Week,Month_of_Travel,Holiday_Season,Demand,Weather_Conditions,Passenger_Count,Promotion_Type,Fuel_Price,Flight_Price,Promotion_Type_Missing
0,1,Airline B,26290,21253,8286.0,8:23,20:19,11.94,Boeing 787,0,Wednesday,December,Summer,Low,Rain,240,Special Offer,0.91,643.93,0
1,2,Airline C,11232,11253,2942.0,20:28,1:45,5.29,Airbus A320,0,Wednesday,March,Spring,Low,Rain,107,NaN,1.08,423.13,1
2,3,Airline B,1634,1242,2468.0,11:30,15:54,4.41,Boeing 787,1,Sunday,September,Summer,High,Cloudy,131,NaN,0.52,442.17,1
3,4,Missing,1847,29887,3145.0,20:24,1:21,4.96,Boeing 787,0,Sunday,February,Fall,Low,Cloudy,170,Discount,0.71,394.42,0
4,5,Airline B,28434,8060,5558.0,21:59,6:04,8.09,Boeing 737,1,Thursday,January,Missing,Missing,Clear,181,NaN,1.09,804.35,1


In [11]:
# Extract hour of departure and arrival from 'Departure_Time' and 'Arrival_Time'
# Convert 'Departure_Time' and 'Arrival_Time' to datetime objects
data['Departure_Time'] = pd.to_datetime(data['Departure_Time'], format='%H:%M')
data['Arrival_Time'] = pd.to_datetime(data['Arrival_Time'], format='%H:%M')

In [12]:
# Extract hour and minute features from 'Departure_Time' and 'Arrival_Time'
data['Departure_Hour'] = data['Departure_Time'].dt.hour
data['Departure_Minute'] = data['Departure_Time'].dt.minute
data['Arrival_Hour'] = data['Arrival_Time'].dt.hour
data['Arrival_Minute'] = data['Arrival_Time'].dt.minute

In [13]:
# Drop the original 'Departure_Time' and 'Arrival_Time' columns
data.drop(['Departure_Time', 'Arrival_Time'], axis=1, inplace=True)

In [14]:
# Convert categorical variables to numerical using label encoding
label_encoders = {}
categorical_columns = ['Aircraft_Type', 'Day_of_Week', 'Month_of_Travel', 'Demand','Holiday_Season', 'Weather_Conditions', 'Promotion_Type']

data = pd.get_dummies(data, columns=['Airline'], drop_first=True)

for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [15]:
# Scaling/Normalizing numerical features
scaler = StandardScaler()
numeric_columns = ['Distance', 'Duration', 'Number_of_Stops', 'Fuel_Price', 'Passenger_Count']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Flight_ID               45000 non-null  int32  
 1   Departure_City          45000 non-null  int64  
 2   Arrival_City            45000 non-null  int64  
 3   Distance                45000 non-null  float64
 4   Duration                45000 non-null  float64
 5   Aircraft_Type           45000 non-null  int32  
 6   Number_of_Stops         45000 non-null  float64
 7   Day_of_Week             45000 non-null  int32  
 8   Month_of_Travel         45000 non-null  int32  
 9   Holiday_Season          45000 non-null  int32  
 10  Demand                  45000 non-null  int32  
 11  Weather_Conditions      45000 non-null  int32  
 12  Passenger_Count         45000 non-null  float64
 13  Promotion_Type          45000 non-null  int32  
 14  Fuel_Price              45000 non-null

In [16]:

# Feature Selection using SelectKBest and f_regression
X = data.drop(['Flight_Price'], axis=1)
y = data['Flight_Price']

selector = SelectKBest(score_func=f_regression, k=10)  # You can adjust 'k' as needed
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)

In [17]:
# Get the names of selected features
selected_features = X.columns[selected_feature_indices]

In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.3, random_state=42)

In [19]:
# Initialize and train the XGBoost Regressor model
xgb_regressor = XGBRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}
scorer = make_scorer(mean_absolute_error, greater_is_better=False)
random_search = RandomizedSearchCV(
    xgb_regressor,
    param_distributions=param_grid,
    scoring=scorer,
    cv=5,  # You can adjust the number of cross-validation folds
    n_iter=50,  # Adjust the number of iterations as needed
    verbose=0,  # Increase verbosity for progress updates
    n_jobs=-1,  # Utilize all available CPU cores
    random_state=42,  # Set a random seed for reproducibility
)


In [20]:
# Fit the random search to the data
random_search.fit(X_train, y_train)

best_xgb_model = random_search.best_estimator_
best_hyperparameters = random_search.best_params_

print(best_xgb_model)
print(best_hyperparameters)

y_pred = best_xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Best Model MAE: {mae}')
print(f'Best Model RMSE: {rmse}')

test = pd.read_csv("data/test.csv")

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)
{'subsample': 1.0, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best Model MAE: 9.720765587384259
Best Model RMSE: 13.169900073677315


In [21]:
# Handle missing values
categorical_columns = ['Airline', 'Departure_City', 'Arrival_City', 'Demand', 'Aircraft_Type', 'Day_of_Week', 'Month_of_Travel','Holiday_Season','Weather_Conditions']

for column in categorical_columns:
    test[column].fillna('Missing', inplace=True)

numerical_columns = ['Distance', 'Fuel_Price']

for column in numerical_columns:
    test[column].fillna(test[column].median(), inplace=True)

data['Promotion_Type_Missing'] = test['Promotion_Type'].isnull().astype(int)

data['Fuel_Price'].fillna(test['Fuel_Price'].median(), inplace=True)

unique_cities = set(test['Departure_City'].unique()) | set(test['Arrival_City'].unique())
city_mapping = {city: label for label, city in enumerate(unique_cities)}
test['Departure_City'] = test['Departure_City'].map(city_mapping)
test['Arrival_City'] = test['Arrival_City'].map(city_mapping)
test['Flight_ID'] = test['Flight_ID'].str.extract('(\d+)').astype(int)

In [22]:
# Extract hour of departure and arrival from 'Departure_Time' and 'Arrival_Time'
# Convert 'Departure_Time' and 'Arrival_Time' to datetime objects
test['Departure_Time'] = pd.to_datetime(test['Departure_Time'],format='%H:%M')
test['Arrival_Time'] = pd.to_datetime(test['Arrival_Time'],format='%H:%M')

In [23]:
# Extract hour and minute features from 'Departure_Time' and 'Arrival_Time'
test['Departure_Hour'] = test['Departure_Time'].dt.hour
test['Departure_Minute'] = test['Departure_Time'].dt.minute
test['Arrival_Hour'] = test['Arrival_Time'].dt.hour
test['Arrival_Minute'] = test['Arrival_Time'].dt.minute

In [24]:
# Drop the original 'Departure_Time' and 'Arrival_Time' columns
test.drop(['Departure_Time', 'Arrival_Time'], axis=1, inplace=True)


In [25]:
# Convert categorical variables to numerical using label encoding
label_encoders = {}
categorical_columns = ['Aircraft_Type', 'Day_of_Week', 'Month_of_Travel', 'Demand','Holiday_Season', 'Weather_Conditions', 'Promotion_Type']

test = pd.get_dummies(test, columns=['Airline'], drop_first=True)

for column in categorical_columns:
    le = LabelEncoder()
    test[column] = le.fit_transform(test[column])
    label_encoders[column] = le


In [26]:
# Scaling/Normalizing numerical features
scaler = StandardScaler()
numeric_columns = ['Distance', 'Duration', 'Number_of_Stops', 'Fuel_Price', 'Passenger_Count']
test[numeric_columns] = scaler.fit_transform(test[numeric_columns])
test = test[['Distance', 'Duration', 'Aircraft_Type', 'Number_of_Stops',
       'Day_of_Week', 'Month_of_Travel', 'Holiday_Season', 'Demand',
       'Weather_Conditions', 'Fuel_Price']]
test.head()

y_pred = best_xgb_model.predict(test)

In [27]:
# Create a DataFrame with 'Flight_ID' and 'Predicted_Price' columns
submission_df = pd.DataFrame({'Flight_ID': test.index, 'Flight_Price': y_pred})


In [28]:
# Save the DataFrame to a CSV file
submission_df.to_csv('data/Submission.csv', index=False)